Prepare the data:

In [5]:
import pandas

df = pandas.read_csv("E:/download/MSR_data_cleaned/MSR_data_cleaned.csv")

# print(df.sample(1))
print(df.columns)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50144\2201264182.py:3: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv("E:/download/MSR_data_cleaned/MSR_data_cleaned.csv")


Index(['Unnamed: 0', 'Access Gained', 'Attack Origin',
       'Authentication Required', 'Availability', 'CVE ID', 'CVE Page',
       'CWE ID', 'Complexity', 'Confidentiality', 'Integrity',
       'Known Exploits', 'Publish Date', 'Score', 'Summary', 'Update Date',
       'Vulnerability Classification', 'add_lines', 'codeLink', 'commit_id',
       'commit_message', 'del_lines', 'file_name', 'files_changed',
       'func_after', 'func_before', 'lang', 'lines_after', 'lines_before',
       'parentID', 'patch', 'project', 'project_after', 'project_before',
       'vul', 'vul_func_with_fix'],
      dtype='object')


In [25]:
import pandas as pd
import random

# Load the dataset
file_path = "E:/download/MSR_data_cleaned/MSR_data_cleaned.csv"
df = pd.read_csv(file_path)

# Select samples for each class
vulnerable_samples = df[df['vul'] == 1]
non_vulnerable_samples = df[df['vul'] == 0]

# Function to create a prompt for In-Context Learning
def create_prompt(num_samples=4):
    # Randomly select samples for the prompt
    selected_vulnerable = vulnerable_samples.sample(num_samples // 2, random_state=42)
    selected_non_vulnerable = non_vulnerable_samples.sample(num_samples // 2, random_state=42)
    
    # Combine samples and shuffle them
    prompt_samples = pd.concat([selected_vulnerable, selected_non_vulnerable]).sample(frac=1, random_state=42)

    hidden_label_sample = prompt_samples.sample(1, random_state=random.randint(0,1000))
    prompt_samples = prompt_samples.drop(hidden_label_sample.index)
    # Format the prompt
    prompt = ""
    for _, row in prompt_samples.iterrows():
        func = row['func_before']
        label = "Yes" if row['vul'] == 1 else "No"
        print(label)
        
        prompt += f"Function:\n{func}\nVulnerable: {label}\n\n"
        

    # Randomly choose another function to ask GPT
    
    question_func = hidden_label_sample.iloc[0]['func_before']

    # Final prompt to send to GPT
    final_prompt = prompt + f"Function:\n{question_func}\nVulnerable:?  based on the samples. Please identify if this func is vulnerable.  only answer Yes or No"

    return final_prompt, hidden_label_sample.iloc[0]['vul']

# Create and display the prompt
for i in range(10):
    ic_prompt, real_label = create_prompt()
    print(real_label)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50144\1837465298.py:6: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Yes
No
No
1
Yes
Yes
No
0
Yes
No
No
1
Yes
No
No
1
Yes
No
Yes
0
No
Yes
No
1
Yes
Yes
No
0
Yes
No
Yes
0
Yes
No
No
1
No
Yes
No
1


In [28]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import accuracy_score, f1_score
import requests
import os

# Set your OpenAI API key
# openai.api_key = "sk-tJcYK0JRi38t07XVzqphbgRClcMjYTtYx1Fvvr1sYfT3BlbkFJ60-ERcu96EWA7pifLV0y67uFF0jpgV9PiNU4mrT44A"
client = OpenAI()
# Load the dataset
file_path = "E:/download/MSR_data_cleaned/MSR_data_cleaned.csv"
df = pd.read_csv(file_path)
os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"
 
# Select samples for each class
vulnerable_samples = df[df['vul'] == 1]
non_vulnerable_samples = df[df['vul'] == 0]

def create_prompt(num_samples=4):
    # Randomly select samples for the prompt
    selected_vulnerable = vulnerable_samples.sample(num_samples // 2, random_state=42)
    selected_non_vulnerable = non_vulnerable_samples.sample(num_samples // 2, random_state=42)
    
    # Combine samples and shuffle them
    prompt_samples = pd.concat([selected_vulnerable, selected_non_vulnerable]).sample(frac=1, random_state=42)

    hidden_label_sample = prompt_samples.sample(1, random_state=random.randint(0,1000))    
    prompt_samples = prompt_samples.drop(hidden_label_sample.index)
    # Format the prompt
    prompt = ""
    for _, row in prompt_samples.iterrows():
        func = row['func_before']
        label = "Yes" if row['vul'] == 1 else "No"
        
        prompt += f"Function:\n{func}\nVulnerable: {label}\n\n"
        

    # Randomly choose another function to ask GPT
    
    question_func = hidden_label_sample.iloc[0]['func_before']

    # Final prompt to send to GPT
    final_prompt = prompt + f"Function:\n{question_func}\nVulnerable:?  based on the samples. Please identify if this func is vulnerable.  only answer Yes or No"

    return final_prompt, hidden_label_sample.iloc[0]['vul']

def query_chatgpt(prompt):
    try:
        messages = [
            {"role": "user", "content": prompt}
        ]
        # Send prompt to ChatGPT
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Adjust the engine name as needed
            messages=messages,
            max_tokens=4000
            
        )
        # Extract the response text
        return response.choices[0].message.content
    except requests.exceptions.Timeout:
        print("Request timed out. Skipping this part.")
        return ""

# Metrics to evaluate
true_labels = []
predicted_labels = []


for i in range(1500):
    # Generate a prompt
    prompt, true_label = create_prompt()
    
    print("creating prompt...")
    
    # Query ChatGPT and get the response
    response = query_chatgpt(prompt)
    
    print("get response from gpt...")
    print(true_label)
    print(response)
    if response is None:
        continue  # Skip if token limit is exceeded

    # Convert GPT response to label (assuming "Yes" -> 1, "No" -> 0)
    predicted_label = 1 if "yes" in response.lower() else 0

    # Store the true and predicted labels
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

    # Print accuracy and F1 score every 10 iterations
    if (i + 1) % 10 == 0:
        accuracy = accuracy_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)
        print(f"Iteration {i + 1}: Accuracy = {accuracy:.2f}, F1 Score = {f1:.2f}")
print(true_labels)
print("------------------------------------------")
print(predicted_labels)
# Final Accuracy and F1 Score after 200 iterations
final_accuracy = accuracy_score(true_labels, predicted_labels)
final_f1 = f1_score(true_labels, predicted_labels)
print(f"Final Accuracy: {final_accuracy:.2f}, Final F1 Score: {final_f1:.2f}")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50144\456703921.py:12: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


creating prompt...
get response from gpt...
0
Yes
creating prompt...
get response from gpt...
0
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
0
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
0
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
0
No
creating prompt...
get response from gpt...
1
No
Iteration 10: Accuracy = 0.40, F1 Score = 0.00
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
0
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
0
Yes
creating prompt...
get response from gpt...
1
No
creating prompt...
get response from gpt...
0
No
creating prompt...
g

This is for few shots

In [30]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import accuracy_score, f1_score
import random
import os

# Initialize OpenAI API client
client = OpenAI()

# Load the dataset
file_path = "E:/download/MSR_data_cleaned/MSR_data_cleaned.csv"
df = pd.read_csv(file_path)
os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

# Select samples for each class
vulnerable_samples = df[df['vul'] == 1]
non_vulnerable_samples = df[df['vul'] == 0]

def create_prompt(num_shots=3):
    # Randomly select samples for context examples
    selected_vulnerable = vulnerable_samples.sample(num_shots // 2, random_state=42)
    selected_non_vulnerable = non_vulnerable_samples.sample(num_shots // 2, random_state=42)
    
    # Combine samples and shuffle them
    prompt_samples = pd.concat([selected_vulnerable, selected_non_vulnerable]).sample(frac=1, random_state=42)

    # Randomly choose one sample to hide the label
    hidden_label_sample = prompt_samples.sample(1, random_state=random.randint(0, 1000))
    prompt_samples = prompt_samples.drop(hidden_label_sample.index)

    # Format the prompt for in context learning
    prompt = "Below are some code functions and their vulnerability status:\n\n"
    for _, row in prompt_samples.iterrows():
        func = row['func_before']
        label = "Yes" if row['vul'] == 1 else "No"
        prompt += f"Function:\n{func}\nVulnerable: {label}\n\n"

    # Add the sample with the hidden label
    question_func = hidden_label_sample.iloc[0]['func_before']
    final_prompt = prompt + f"Function:\n{question_func}\nVulnerable:"

    return final_prompt, hidden_label_sample.iloc[0]['vul']

def query_chatgpt(prompt):
    try:
        # Send prompt to ChatGPT
        response = client.chat.completions.create(
            model="gpt-4",  # Adjust the engine name as needed
            messages=[{"role": "user", "content": prompt}],
            max_tokens=4000
        )
        # Extract the response text
        return response.choices[0].message.content
    except requests.exceptions.Timeout:
        print("Request timed out. Skipping this part.")
        return ""

# Metrics to evaluate
true_labels = []
predicted_labels = []

# Perform 200 iterations
for i in range(200):
    # Generate a prompt
    prompt, true_label = create_prompt()
    print("Creating prompt...")

    # Query ChatGPT and get the response
    response = query_chatgpt(prompt)
    print("Got response from GPT...")

    if response is None:
        continue  # Skip if token limit is exceeded

    # Convert GPT response to label (assuming "Yes" -> 1, "No" -> 0)
    predicted_label = 1 if "yes" in response.lower() else 0

    # Store the true and predicted labels
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

    # Print accuracy and F1 score every 10 iterations
    if (i + 1) % 10 == 0:
        accuracy = accuracy_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)
        print(f"Iteration {i + 1}: Accuracy = {accuracy:.2f}, F1 Score = {f1:.2f}")

# Final Accuracy and F1 Score after 200 iterations
final_accuracy = accuracy_score(true_labels, predicted_labels)
final_f1 = f1_score(true_labels, predicted_labels)
print(f"Final Accuracy: {final_accuracy:.2f}, Final F1 Score: {final_f1:.2f}")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50144\557698223.py:12: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Iteration 10: Accuracy = 0.90, F1 Score = 0.86
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Creating prompt...
Got response from GPT...
Iteration 20: Accuracy = 0.85, F1 Score = 0.80
Creating prompt...
Got res

In [35]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import accuracy_score, f1_score
import requests
import os
import random

# Set your OpenAI API key
client = OpenAI()

# Load the dataset
file_path = "E:/download/MSR_data_cleaned/MSR_data_cleaned.csv"
df = pd.read_csv(file_path)
os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"
 
# Select samples with label 0 (non-vulnerable)
non_vulnerable_samples = df[df['vul'] == 0]
vulnerable_samples = df[df["vul"] == 1]

def create_zero_shot_prompt():
    # Randomly select a sample with label 0 to hide its label and let GPT predict
    i = random.randint(0,100)
    if i % 2 == 0:
        sample = vulnerable_samples.sample(1, random_state=42).iloc[0]
    else:
        sample = non_vulnerable_samples.sample(1, random_state=42).iloc[0]
    
    func = sample['func_before']

    # Create a zero-shot prompt
    prompt = f"""
    Given the following function, determine whether it is vulnerable or not.

    Function:
    {func}

    Is this function vulnerable? only answer yes or no
    """
    
    return prompt, sample['vul']

def query_chatgpt(prompt):
    try:
        # Send prompt to ChatGPT
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Adjust the engine name as needed
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100
        )
        # Extract the response text
        return response.choices[0].message.content.strip()
    except requests.exceptions.Timeout:
        print("Request timed out. Skipping this part.")
        return ""
    

# Metrics to evaluate
true_labels = []
predicted_labels = []

# Perform 200 iterations
for i in range(200):
    # Generate a zero-shot prompt
    prompt, true_label = create_zero_shot_prompt()
    print("Creating zero-shot prompt...")
    print(true_label)
    # Query ChatGPT and get the response
    response = query_chatgpt(prompt)
    print("Received response from GPT...")

    if response is None or len(response) == 0:
        continue  # Skip if no response or invalid

    # Convert GPT response to label (assuming "Yes" -> 1, "No" -> 0)
    predicted_label = 1 if "yes" in response.lower() else 0
    print(predicted_label)
    # Store the true and predicted labels
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

    # Print accuracy and F1 score every 10 iterations
    if (i + 1) % 10 == 0:
        accuracy = accuracy_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)
        print(f"Iteration {i + 1}: Accuracy = {accuracy:.2f}, F1 Score = {f1:.2f}")

# Final Accuracy and F1 Score after 200 iterations
final_accuracy = accuracy_score(true_labels, predicted_labels)
final_f1 = f1_score(true_labels, predicted_labels)
print(f"Final Accuracy: {final_accuracy:.2f}, Final F1 Score: {final_f1:.2f}")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50144\1453967897.py:13: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Creating zero-shot prompt...
1
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Creating zero-shot prompt...
0
Received response from GPT...
0
Creating zero-shot prompt...
0
Received response from GPT...
0
Creating zero-shot prompt...
0
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Creating zero-shot prompt...
0
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Iteration 10: Accuracy = 0.40, F1 Score = 0.00
Creating zero-shot prompt...
0
Received response from GPT...
0
Creating zero-shot prompt...
0
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Creating zero-shot prompt...
1
Received response from GPT...
0
Creating